# Market Overlap

### SCENARIO

The scenario is: A ficticious company named 'Alpha' is interested in acquiring another ficticious company named 'Beta'. They both belong to the hotelling industry, more specifically they are marketplaces for hotels to announce their accomodation details as well as handle all the booking process with travellers.

### UNDERSTANDING THE COMPANIES

__Alpha__ offer hotels in destinations worldwide. They've entered Brazil's market recently and have been investing in Facebook and Google Ads to get more hotels to publish in their marketplace. However, the CAC (customer acquisition cost) to get more hotels on board is too high and after several weeks trying to improve performance Alpha's board have decided to look for other options of increasing the number of hotels in their marketplace.

__Beta__ on the other hand, operates only in Brazil and although has a smaller scale than Alpha overall, it has a considerable amount of hotels already on board and operating.

### WHY THE MARKET OVERLAP ANALYSIS

Seeking a way to reduce the CAC, Alpha has made a move to buy Beta with all it's hotels and marketplace service. Even though Beta has shown interest on the deal, Alpha still want's to look at some real data in order to reach the conclusion if the CAC would really be lower than their current marketing investments.

As of now, both companies does not know if their hotels are unique to them or actually common between both, as its a common practice among hotels to publish their accomodations in different marketplaces.

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
dtype = {
    'registration_id': str
}

parse_dates = [
    'registration_date',
    'latest_booking'
]

alpha = pd.read_csv(
    'alpha_hotels.csv',
    dtype=dtype,
    parse_dates=parse_dates
)

In [60]:
dtype = {
    'Registration Id': str
}

parse_dates = [
    'Registration Date',
    'Last Booking'
]

beta = pd.read_csv(
    'Beta Hotels.csv',
    dtype=dtype,
    parse_dates=parse_dates
)

In [61]:
alpha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1843 entries, 0 to 1842
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1843 non-null   int64         
 1   hotel_name         1843 non-null   object        
 2   address            1843 non-null   object        
 3   city               1843 non-null   object        
 4   country            1843 non-null   object        
 5   registration_date  1843 non-null   datetime64[ns]
 6   latest_booking     1843 non-null   datetime64[ns]
 7   total_bookings     1843 non-null   int64         
 8   registration_id    1843 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 129.7+ KB


In [62]:
beta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2601 entries, 0 to 2600
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Id                 2601 non-null   int64         
 1   Hotel Name         2601 non-null   object        
 2   Address            2601 non-null   object        
 3   City State         2601 non-null   object        
 4   Registration Date  2601 non-null   datetime64[ns]
 5   Last Booking       2601 non-null   datetime64[ns]
 6   Total Bookings     2601 non-null   int64         
 7   Registration Id    2601 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 162.7+ KB
